In [2]:
import numpy as np
import pandas as pd

In [3]:
#df = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\around_station_small.parquet')
#df = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\2018_part_0_small_ALL_data_with_lightning.parquet')
df = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\around_station_2018_part_3_of_6_small.parquet')

In [4]:
print(df.shape)

(137172, 25)


In [5]:
df.head(6)

,File_Name,Cloud_Number,Area,Center_lonlat_1,Center_lonlat_2,Eccentricity,Orientation,TBBmax,TBBmean,TBBmedian,...,Minute,Second,Latitude,Longitude,Error of location,Number of discharges,Number of day,Lightning is in cloud,Lightning is in ellipse,TBB near lightning
0,20180501_0930,1107,474.714291,40.995533,37.647865,0.833665,13.722172,-32.010998,-36.487572,-35.158001,...,18.0,15.470687,37.7383,40.9038,12.7,5.0,737181.387679,0.0,1.0,-37.553005
1,20180501_0930,1107,474.714291,40.995533,37.647865,0.833665,13.722172,-32.010998,-36.487572,-35.158001,...,20.0,32.790843,37.5734,41.0306,20.9,7.0,737181.389268,0.0,1.0,-33.460995
2,20180501_0930,1107,474.714291,40.995533,37.647865,0.833665,13.722172,-32.010998,-36.487572,-35.158001,...,20.0,33.086162,37.5874,40.9316,7.8,7.0,737181.389272,0.0,1.0,-39.820995
3,20180501_0930,1107,474.714291,40.995533,37.647865,0.833665,13.722172,-32.010998,-36.487572,-35.158001,...,20.0,54.272556,37.5728,41.0188,20.6,7.0,737181.389517,0.0,1.0,-35.158001
4,20180501_0930,1107,474.714291,40.995533,37.647865,0.833665,13.722172,-32.010998,-36.487572,-35.158001,...,20.0,54.406711,37.5813,40.9499,6.8,5.0,737181.389519,0.0,1.0,-39.820995
5,20180501_0930,1107,474.714291,40.995533,37.647865,0.833665,13.722172,-32.010998,-36.487572,-35.158001,...,21.0,24.940198,37.5872,40.9433,15.3,6.0,737181.389872,0.0,1.0,-39.820995


In [6]:
print(df.columns)

Index(['File_Name', 'Cloud_Number', 'Area', 'Center_lonlat_1',
       'Center_lonlat_2', 'Eccentricity', 'Orientation', 'TBBmax', 'TBBmean',
       'TBBmedian', 'TBBmin', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second', 'Latitude', 'Longitude', 'Error of location',
       'Number of discharges', 'Number of day', 'Lightning is in cloud',
       'Lightning is in ellipse', 'TBB near lightning'],
      dtype='object')


## Number of discharges per minute, by WWLLN

In [7]:
import pandas as pd

# Assuming df is already loaded
# Convert time-related columns to integer values
df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']] = df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']].astype(int)

# Create a full datetime column from these components
df['Timestamp'] = pd.to_datetime(
    df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']]
)

# Aggregate by minute by flooring the timestamp to the nearest minute
df['Minute_Timestamp'] = df['Timestamp'].dt.floor('T')  # 'T' for minute

# Count the number of discharges for each minute
minute_discharge_counts = df.groupby('Minute_Timestamp').size().reset_index(name='Discharge_Count')

# Print the table
print(minute_discharge_counts)



         Minute_Timestamp  Discharge_Count
0     2018-05-01 09:18:00                1
1     2018-05-01 09:20:00                4
2     2018-05-01 09:21:00                1
3     2018-05-01 09:22:00                1
4     2018-05-01 09:23:00                1
...                   ...              ...
29205 2018-06-30 02:45:00                1
29206 2018-06-30 02:46:00                1
29207 2018-06-30 02:47:00                2
29208 2018-06-30 03:48:00                2
29209 2018-06-30 04:02:00                2

[29210 rows x 2 columns]


## Number of discharges per minute, by Lightning Detector (2) (local system)

In [8]:
df_csv = pd.read_csv('F:/2024_Lightning_Analysis/Aragats_measurements/2018/2018_Lightning_Detector_1_5_km.csv')

In [9]:
df_csv.columns

Index(['Date/Time', ' CG+  Radius 5 km', ' CG-  Radius 5 km',
       ' IC+  Radius 5 km', ' IC-  Radius 5 km'],
      dtype='object')

### Table "minute - Number of discharges" for Lightning Detector (2) csv

In [10]:
# Convert the "Date/Time" column in CSV to datetime format
df_csv['Timestamp'] = pd.to_datetime(df_csv['Date/Time'], format='%d-%b-%y %H:%M:%S.%f')
df_csv['Minute_Timestamp'] = df_csv['Timestamp'].dt.floor('T')

# Sum the four columns for total discharges per minute in CSV data
df_csv['Total_Discharge_Count'] = df_csv[[' CG+  Radius 5 km', ' CG-  Radius 5 km', ' IC+  Radius 5 km', ' IC-  Radius 5 km']].sum(axis=1)

# Sum only the 'CG' columns for CG-specific discharges per minute in CSV data
df_csv['CG_Discharge_Count'] = df_csv[[' CG+  Radius 5 km', ' CG-  Radius 5 km']].sum(axis=1)

# Group by minute to aggregate the counts
csv_minute_discharge_counts = df_csv.groupby('Minute_Timestamp')[['Total_Discharge_Count', 'CG_Discharge_Count']].sum().reset_index()

# Print the resulting table
print("CSV Data (Minute - Total Discharges - CG Discharges):")
print(csv_minute_discharge_counts)


CSV Data (Minute - Total Discharges - CG Discharges):
         Minute_Timestamp  Total_Discharge_Count  CG_Discharge_Count
0     2018-01-01 19:51:00                      0                   0
1     2018-01-01 19:52:00                      0                   0
2     2018-01-01 21:39:00                      0                   0
3     2018-01-01 23:04:00                      0                   0
4     2018-01-02 09:58:00                      0                   0
...                   ...                    ...                 ...
23758 2018-12-27 23:12:00                      0                   0
23759 2018-12-27 23:13:00                      0                   0
23760 2018-12-27 23:16:00                      0                   0
23761 2018-12-27 23:18:00                      0                   0
23762 2018-12-28 12:58:00                      0                   0

[23763 rows x 3 columns]


### Table "minute - Number of discharges" for WWLLN and Lightning Detector (2)

In [19]:
# Convert time-related columns in parquet data to integer
df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']] = df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']].astype(int)

# Create a full timestamp for the parquet data
df['Timestamp'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])
df['Minute_Timestamp'] = df['Timestamp'].dt.floor('T')  # Floor to nearest minute
parquet_minute_discharge_counts = df.groupby('Minute_Timestamp').size().reset_index(name='Discharge_Count')

In [20]:
# Step 1: Merge parquet (WWLLN) data with CSV (Lightning Detector) data by minute
combined_discharge_counts = pd.merge(
    parquet_minute_discharge_counts,
    csv_minute_discharge_counts,
    on='Minute_Timestamp',
    how='inner'
)

# Step 2: Rename columns for clarity
combined_discharge_counts.columns = ['Minute_Timestamp', 'WWLLN_Discharge_Count', 'LD_Total_Discharge_Count', 'LD_CG_Discharge_Count']

# Step 3: Print the combined table
print("Minute - WWLLN Discharge Count - LD Total Discharge Count - LD CG Discharge Count:")
print(combined_discharge_counts)


Minute - WWLLN Discharge Count - LD Total Discharge Count - LD CG Discharge Count:
        Minute_Timestamp  WWLLN_Discharge_Count  LD_Total_Discharge_Count  \
0    2018-05-02 12:54:00                      3                         0   
1    2018-05-03 11:00:00                     14                         0   
2    2018-05-03 14:56:00                      2                         0   
3    2018-05-03 14:58:00                      1                         0   
4    2018-05-03 14:59:00                      1                         0   
...                  ...                    ...                       ...   
6662 2018-06-25 23:59:00                      2                         0   
6663 2018-06-26 00:03:00                      1                         0   
6664 2018-06-26 00:04:00                      1                         0   
6665 2018-06-26 00:14:00                      1                         0   
6666 2018-06-26 00:40:00                      1                       

In [22]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add traces for each data series
fig.add_trace(go.Scatter(
    x=combined_discharge_counts['Minute_Timestamp'], 
    y=combined_discharge_counts['WWLLN_Discharge_Count'],
    mode='lines+markers',
    name='WWLLN Discharge Count',
    line=dict(color='blue'),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=combined_discharge_counts['Minute_Timestamp'], 
    y=combined_discharge_counts['LD_Total_Discharge_Count'],
    mode='lines+markers',
    name='LD Total Discharge Count',
    line=dict(color='green'),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=combined_discharge_counts['Minute_Timestamp'], 
    y=combined_discharge_counts['LD_CG_Discharge_Count'],
    mode='lines+markers',
    name='LD CG Discharge Count',
    line=dict(color='red'),
    marker=dict(size=6)
))

# Update layout for labels, title, and legend
fig.update_layout(
    title="Discharges per Minute - WWLLN and Lightning Detector data",
    xaxis_title="Minute",
    yaxis_title="Number of Discharges",
    legend_title="Data Source",
    xaxis=dict(tickangle=45),
    height=600,  # Adjust height for better readability
    width=1000   # Adjust width for better readability
)

# Show the interactive plot
fig.show()